# Установка необходимых библиотек

In [ ]:
!pip install -r requirements.txt

In [9]:
import pandas as pd
import numpy as np
import kagglehub
import os
import pickle
import faiss
import asyncio
import nest_asyncio
from sentence_transformers import SentenceTransformer
from telegram import Update, ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove
from telegram.ext import (ApplicationBuilder, ContextTypes, CommandHandler, MessageHandler, filters)

from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage

# Знакомство с данными

In [10]:
# Скачиваем датасет с KaggleHub
path = kagglehub.dataset_download("coolonce/recipes-and-interpretation-dim")

# Собираем путь к нужному файлу
file_path = os.path.join(path, 'all_recepies_inter.csv')

# Загружаем CSV-файл
df = pd.read_csv(file_path, sep='\t')
print(df.head())

   Unnamed: 0                                               name  \
0           0  рассольник классический с перловкой и солеными...   
1           1                    Суп пюре из белокочаной капусты   
2           2                     Постные щи из квашеной капусты   
3           3                  Тюря- простой суп быстро и вкусно   
4           4                    Фасолевый суп из красной фасоли   

                                         composition         cooking_type  \
0  [{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...          варка,жарка   
1  [{'Капуста белокочанная': 50.0, 'unit': 'гр'},...                варка   
2  [{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...  варка,жарка,тушение   
3  [{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...                сырое   
4  [{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...                варка   

                                          Инструкции dish_type        Дата  \
0  Подготовить указанные ингредиенты для приготов.

In [11]:
df.info() # Проверяем наличие пропусков и знакомимся с полями и типами данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27884 entries, 0 to 27883
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         27884 non-null  int64 
 1   name               27884 non-null  object
 2   composition        27884 non-null  object
 3   cooking_type       27884 non-null  object
 4   Инструкции         27884 non-null  object
 5   dish_type          27884 non-null  object
 6   Дата               27884 non-null  object
 7   photo              27884 non-null  object
 8   source             27884 non-null  object
 9   composition_inter  27884 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.1+ MB


# Предобработка

In [12]:
df = df.drop(columns=['Unnamed: 0', 'Дата', 'source', 'composition_inter', 'cooking_type']) # Удаляем лишние колонки

In [13]:
df.columns

Index(['name', 'composition', 'Инструкции', 'dish_type', 'photo'], dtype='object')

In [14]:
df.columns = ['Название блюда', 'Ингредиенты', 'Приготовление', 'Тип блюда', 'Фото'] # Переименовываем колонки для удобства на русский язык

In [15]:
# Удаление дубликатов, сохраняя только первое вхождение
df_cleaned = df.drop_duplicates()

Вывод: датасет из 27 тысяч рецептов, содержащий в себе название, ингридиенты, процесс приготовления, фото и ссылку на рецепт. Для дальнейшей обработки оставляем только ключевые поля, содержащие важную информацию и убираем дубликаты.

# Векторизация и индексация

In [16]:
# Загружаем энкодер для преобразования текста в эмбеддинги
encoder = SentenceTransformer("intfloat/multilingual-e5-base")

# Формируем тексты из DataFrame: название, ингредиенты, шаги, тип блюда
texts = (
    'Название блюда: ' + df['Название блюда'] +
    '. Ингредиенты: ' + df['Ингредиенты'] +
    '. Приготовление: ' + df['Приготовление'] +
    '. Тип блюда: ' + df['Тип блюда']
).tolist()

# Добавляем префикс "passage: " — формат, ожидаемый моделью E5
texts_for_index = ["passage: " + t for t in texts]

# Преобразуем тексты в эмбеддинги с помощью модели
embeddings = encoder.encode(texts_for_index, show_progress_bar=True, batch_size=32)

# Определяем размерность эмбеддингов
dim = embeddings[0].shape[0]

# Создаём FAISS-индекс для поиска по L2 (евклидовому) расстоянию
index = faiss.IndexFlatL2(dim)

# Добавляем эмбеддинги в индекс
index.add(np.array(embeddings))

# Сохраняем FAISS-индекс в бинарный файл
faiss.write_index(index, "faiss_index.bin")

# Сохраняем тексты, к которым относятся эмбеддинги (например, для вывода найденных рецептов)
with open("texts.pkl", "wb") as f:
    pickle.dump(texts, f)

print("Индекс и тексты успешно сохранены.")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/872 [00:00<?, ?it/s]

Индекс и тексты успешно сохранены.


Модель эмбеддингов: intfloat/multilingual-e5-base
* Оптимизирована под Retrieval (поиск): обучена на задаче поиска (retrieval) с парами query и passage. Это значительно повышает точность поиска.
* Поддерживает русский язык.
* Легко запускается на локальной машине.
* Метод индексирования: FAISS.IndexFlatL2

FAISS — стандарт в задачах семантического поиска, совместим с NumPy и легко расширяется.
* IndexFlatL2 — точный индекс (exhaustive search), не использует приближённые методы, а проводит полный перебор по всем векторам. Подходит, если база рецептов относительно небольшая.
* Не требует сложной настройки, можно сразу использовать и легко переключиться на ускоренные варианты  при необходимости.
